In [ ]:
using BenchmarkTools

In [ ]:
include("/home/kwat/github/Kraft.jl/src/Kraft.jl")

In [ ]:
const genes = readlines("/home/kwat/github/kraft/notebook/genes.txt")[2:end]

In [ ]:
const gene_values = fill(
    1 / length(genes),
    length(genes),
)

In [ ]:
const gmt_file_path = [
    "/home/kwat/garden/data/gene_set/msigdb_v6.2/h.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c1.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c2.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c3.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c5.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c6.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c7.all.v6.2.symbols.gmt",
#     "/home/kwat/garden/data/gene_set/hippo_pathway.gmt",
#     "/home/kwat/garden/data/gene_set/stem_cell.gmt",
#     "/home/kwat/garden/data/gene_set/yap_signature.gmt",
]

In [ ]:
const gene_set_name_genes = Kraft.read_gmt(gmt_file_path)

const n_gene_set = length(gene_set_name_genes)

In [ ]:
const gene_set_genes = gene_set_name_genes[sort(collect(keys(gene_set_name_genes)))[1]]

## Benchmark Julia

In [ ]:
@benchmark Kraft.compute_gene_set_enrichment(
    gene_values,
    genes,
    gene_set_genes,
)

In [ ]:
benchmark_result = @benchmark Kraft.compute_gene_set_enrichment(
    gene_values,
    genes,
    gene_set_name_genes,
)

ms_per_gene_set = round(
    minimum(benchmark_result.times) / 1e6 / n_gene_set,
    sigdigits=2,
)

println("$ms_per_gene_set ms / gene set")

benchmark_result

## Benchmark Python

In [ ]:
using PyCall

In [ ]:
pd = pyimport("pandas")

In [ ]:
kraft = pyimport("kraft")

In [ ]:
const gene_value = pd.Series(
    gene_values,
    index=genes,
)

@benchmark kraft.run_single_sample_gsea(
    gene_value,
    gene_set_genes,
    plot=false,
)

In [ ]:
const py_gene_x_sample = gene_value.to_frame()

const py_gmt = kraft.read_gmt(gmt_file_path)

benchmark_result = @benchmark kraft.run_single_sample_gseas(
    py_gene_x_sample,
    py_gmt,
)

ms_per_gene_set = round(
    minimum(benchmark_result.times) / 1e6 / n_gene_set,
    sigdigits=2,
)

println("$ms_per_gene_set ms / gene set")

benchmark_result

## Gene x Sample ==> Gene Set x Sample

In [ ]:
using CSV

In [ ]:
const gene_x_sample = CSV.read(
    "/home/kwat/github/match_pattern/output/gene_x_sample.processed.tsv",
)

In [ ]:
gene_set_x_sample = Kraft.make_gene_set_x_sample(
    gene_x_sample,
    gene_set_name_genes;
    gene_set_x_sample_file_path="/home/kwat/github/match_pattern/output/gene_set_x_sample.tsv",
)